# **四、量子计算云平台量子编程教学**

-----------------------

# 教学大纲
## 4. [高阶应用技巧](#jump_4)
### &nbsp;&nbsp;&nbsp;&nbsp; 4.1 [实验合集管理](#jump_4_1)
### &nbsp;&nbsp;&nbsp;&nbsp; 4.2 [切换量子计算机](#jump_4_2)
### &nbsp;&nbsp;&nbsp;&nbsp; 4.3 [实验提交动作分解](#jump_4_3)
### &nbsp;&nbsp;&nbsp;&nbsp; 4.4 [多次运行同一线路](#jump_4_4)
### &nbsp;&nbsp;&nbsp;&nbsp; 4.5 [线路传参](#jump_4_5)
### &nbsp;&nbsp;&nbsp;&nbsp; 4.6 [概率统计](#jump_4_6)
### &nbsp;&nbsp;&nbsp;&nbsp; 4.7 [读取矫正](#jump_4_7)

# 4 高阶应用技巧<a id="jump_4"></a>

## 4.1 实验合集管理<a id="jump_4_1"></a>

为提供合理的实验归类管理，量子计算云平台提供实验归类管理，提供了实验合集的概念。  
可以在提交实验线路时，将实验线路放在指定的合集下，以便后期管理查看。  
**注意：**
**需要先创建实验合集或将已有合集信息传递到后继实验。  
实验合集之间不能重名，同一合集下实验名称不能重名。  
云平台网站WEB端展示效果待优化，请包涵。**

具体实验合集的创建方式如下：

In [ ]:
from ezQpy import * 
account = Account(login_key='d69a8e1389f2cabde55c3de99fcf8589', machine_name='ClosedBetaQC')

create_res = account.create_experiment('机器学习B')
if create_res == 0:
    print('新建实验合集失败')
else :
    print('新建实验合辑成功，ID=', create_res)
    lab_id = create_res
#实验合集id建议长期保留，以便后继使用。

或者直接使用已有实验ID

In [ ]:
lab_id=Your experimentId
#实验合集id的web查询方式为：在我的实验合集页面，点击需要查看的实验合集，观察浏览器地址栏中信息，eg.
#https://quantumctek-cloud.com/expList.html?experimentId=1639487661985239040
#其中experimentId后的参数即为实验合集id

将量子线路提交到指定的实验集合下的操作方式如下：

In [ ]:
qcis_circuit = '''
H Q7
X Q1
H Q1
CZ Q7 Q1
H Q1
M Q7
M Q1
'''
query_id = account.submit_job(circuit=qcis_circuit, lab_id=lab_id, version="GPT4-a123")
#对于单次提交实验，可以手动输入上述参数，如果想设计自动化程序，version的参数值可以通过计数等格式提前生成。
#其他流程同标准方式。
print(query_id)

## 4.2 切换量子计算机<a id="jump_4_2"></a>

大多数情况，我们都会使用"变量名"代表所需要的量子计算型号，然后在调用account时，直接设置量子计算机。  
对于已经开发完成的程序，利用某台机器(如应答机)验证过的程序，也只需要修改变量名称即可快速修改后继的计算机使用情况。 

In [ ]:
from ezQpy import Account
login_key='d69a8e1389f2cabde55c3de99fcf8589'
machine_name = '应答机A'

account = Account(login_key=login_key, machine_name=machine_name)
#设置用户SDK密钥，选择量子计算机，并创建实例，运行成功后，提示登录成功。
print(account.machine_name)

但有些特殊情况，例如希望在一个程序内，快速提交同一段实验到两个机器上做对比，则可以使用量子计算机的设置功能。

In [ ]:
machine_name = '应答机A'
account = Account(login_key=login_key)#未设置量子计算机
print(account.machine_name)
#查看当前选择的量子计算机
account.set_machine(machine_name) 
#切换量子计算机时，注意最终提交的程序需要遵守量子计算机的拓扑结构限制，否则提交实验时会报错。
print(account.machine_name)
#查看量子计算机设置选择是否生效

## 4.3 实验提交动作分解<a id="jump_4_3"></a>

submit_job()函数实际上是将多个动作封装在一起的函数，如果有特殊需求，可以分布执行提交动作，或者重新设计自己的提交函数。  
当前提交函数内的动作包含：  
#### assign_parameters(circuit, parameters, values)  
将量子线路的变量参数快速带入，生产最终量子线路。  
#### save_experiment(lab_id, exp_data, name='detailtest', language='qcis')  
将量子线路保存到云平台上已被后期调用，返回实验线路id。其中name相当于submit_job中的version变量。
#### run_experiment(exp_id, num_shots=12000)  
运行指定实验id的线路。  
    
所以一个submit_job()提交可以被等效为如下动作。

In [ ]:
qcis_circuit = '''
H Q7
X Q1
H Q1
CZ Q7 Q1
H Q1
M Q7
M Q1
'''
from ezQpy import Account
login_key='d69a8e1389f2cabde55c3de99fcf8589'
machine_name = 'ClosedBetaQC'
account = Account(login_key=login_key, machine_name=machine_name)
#设置用户SDK密钥，选择量子计算机，并创建实例，运行成功后，提示登录成功。

lab_id='d4a7f5f2b6bd4bdc87f2c1e7dc7a3183'
#query_id = account.submit_job(circuit=qcis_circuit, lab_id=lab_id, version="人工智能a1234")
#以上一个submit_job等于以下几个操作
qcis_circuit=account.assign_parameters(circuit=qcis_circuit,parameters=None, values=None)#参数列表和参数值，在上述ubmit_job中采用了默认值NULL。
print(qcis_circuit)
exp_id=account.save_experiment(lab_id=lab_id, exp_data=qcis_circuit, version="人工智能a1234")#language参数在submit_job函数中不可外部输入。当前该值只支持qcis。
print(exp_id)
query_id=account.run_experiment(exp_id=exp_id, num_shots=12000)#num_shots在上述submit_job中采用了默认值12000。
print(query_id)

## 4.4 多次运行同一线路<a id="jump_4_4"></a>

请注意，量子计算云平台只接受固定参数的线路，现阶段即便一些有参数的线路，实际上也都是提前将参数更换为具体数值后，进行提交。所以对同一个线路进行多次，其线路结构，参数数值，都是明确固定的。  
该需求往往用在对特定线路结果的比较，或者直接利用线路结果的应用。  
根据上一节的讲解可知，分步提交线路可以获得线路本身的ID，对其进行保存，在运行时，只要提供该ID即可运行，需要多次运行该线路，只需要提交多次该ID的运行即可。

In [ ]:
#续上例程序
saved_exp_id=exp_id
#saved_exp_id='ustc3142744691105312700000005360'
result_list=[]
for i in range(2) :
    query_id=account.run_experiment(exp_id=saved_exp_id, num_shots=1000)#num_shots在上述submit_job中采用了默认值12000。
    result=account.query_experiment(query_id, max_wait_time=360000)
    result_list.append(result)
print(result_list)

## 4.5 线路传参<a id="jump_4_5"></a>

虽然使用完备的标准门可以实现任意门操作，但对于经典数据的输入，还是希望直接传入到量子程序中，而不是根据参数重新设计线路。  
所以，我们专门在submit_job中直接添加了传参函数assign_parameters。用户也可以在线路准备阶段直接使用该函数进行线路的参数传入。

In [ ]:
#在线路中预置{x}表述待传入参数
qcis_circuit = '''
RX Q1 {n1}
RX Q7 {n2}
H Q1
X Q7
H Q7
CZ Q1 Q7
H Q7
M Q1
M Q7
'''
values=[0.1,0.2]
qcis_circuit=account.assign_parameters(circuit=qcis_circuit, parameters=['n1','n2'], values=values)
print(qcis_circuit)
#或者直接使用submit_job函数直接带入参数
qcis_circuit = '''
RX Q1 {n1}
RX Q7 {n2}
H Q1
X Q7
H Q7
CZ Q1 Q7
H Q7
M Q1
M Q7
'''
lab_id='d4a7f5f2b6bd4bdc87f2c1e7dc7a3183'
query_id = account.submit_job(circuit=qcis_circuit, lab_id=lab_id, version="人工智能1234",parameters=['n1','n2'], values=values)
print(query_id)

上面这种带出参数的意义不大，仅是作为展示。  
在异构程序设计时，会活用这个参数传递，将values根据动态数据实时计算带入，得到更强的参数更新特点。

## 4.6 概率统计<a id="jump_4_6"></a>

为节省带宽，让量子计算机相关服务更专心的做量子计算的服务，当测量比特超过10个时，量子计算机端就不再为用户做结果的概率统计和读取矫正，需要用户自行处理。  
我们提供了基础的概率统计函数，调用方式如下：

In [ ]:
qcis_circuit = '''
H Q7
X Q1
H Q1
CZ Q7 Q1
H Q1
M Q7
M Q1
'''
from ezQpy import * 
account = Account(login_key='d69a8e1389f2cabde55c3de99fcf8589', machine_name='ClosedBetaQC')
query_id = account.submit_job(qcis_circuit)
print(query_id)
if query_id:
    result=account.query_experiment(query_id, max_wait_time=360000)

In [ ]:
probability_whole=account.readout_data_to_state_probabilities_whole(result)
print(probability_whole)
probability_part=account.readout_data_to_state_probabilities_part(result)
print(probability_part)

## 4.7 读取矫正<a id="jump_4_7"></a>

当今的量子计算机在多环节都有保证度问题，在使用结果的统计结果时，需要进行读取矫正。  
注意：该函数的运行，将直接读取量子计算机当前的读取参数值。  
后继将会调整该函数接受历史保持的量子计算机参数。

In [ ]:
probability_cali=account.probability_calibration(result)
print(probability_cali)